# Daily Summary Playground

Use this notebook to experiment with the `DailySummaryService` prompts and LangChain responses without running the production job.

In [2]:
import os
from pathlib import Path

import dotenv

# Load environment variables from .env file
env_path = Path('..') / '.env'
dotenv.load_dotenv(env_path)

# You can also set environment variables directly here
# Uncomment and set your API key if needed:
# os.environ["OPENAI_API_KEY"] = "sk-..."

# Verify OPENAI_API_KEY is loaded
api_key = os.environ.get("OPENAI_API_KEY")
if api_key:
    masked_key = f"{api_key[:7]}...{api_key[-4:]}" if len(api_key) > 11 else "***"
    print(f"✓ OPENAI_API_KEY loaded: {masked_key}")
else:
    print("⚠ WARNING: OPENAI_API_KEY not found in environment")
    print("  Set it via: os.environ['OPENAI_API_KEY'] = 'sk-...'")

✓ OPENAI_API_KEY loaded: sk-proj...-u0A


In [3]:
# Set OPENAI_API_KEY directly if needed (overrides .env file)
# Uncomment and set your key:
# os.environ["OPENAI_API_KEY"] = "sk-..."

# Reload settings to pick up any environment variable changes
import importlib
from app import config
importlib.reload(config)
from app.config import settings

# Verify the API key is available in settings
if settings.openai_api_key:
    masked_key = f"{settings.openai_api_key[:7]}...{settings.openai_api_key[-4:]}" if len(settings.openai_api_key) > 11 else "***"
    print(f"✓ Settings loaded with API key: {masked_key}")
else:
    print("⚠ WARNING: OPENAI_API_KEY not found in settings")
    print("  Set it via: os.environ['OPENAI_API_KEY'] = 'sk-...'")
    print("  Then re-run this cell to reload settings")


✓ Settings loaded with API key: sk-proj...-u0A


In [4]:
import importlib
from datetime import UTC, datetime, timedelta

from app.db.session import SessionLocal
from app.services import daily_summary

# Reload module to pick up any changes (useful during development)
importlib.reload(daily_summary)
from app.services.daily_summary import DailySummaryService

session = SessionLocal()
service = DailySummaryService(session)

In [5]:
# Configure custom window and ticker count
# Set to None to use defaults (previous UTC day)
USE_CUSTOM_WINDOW = False  # Set to True to use custom window below

# Custom time window (UTC-aware datetimes)
# Example: Last 24 hours from now
CUSTOM_WINDOW_START = datetime.now(UTC) - timedelta(days=1)
CUSTOM_WINDOW_END = datetime.now(UTC)

# Maximum number of tickers to include in summary
# Set to None to use default from settings
MAX_TICKERS = 3  # Example: 10

print(f"Configuration:")
print(f"  Use custom window: {USE_CUSTOM_WINDOW}")
if USE_CUSTOM_WINDOW:
    print(f"  Window start: {CUSTOM_WINDOW_START}")
    print(f"  Window end: {CUSTOM_WINDOW_END}")
print(f"  Max tickers: {MAX_TICKERS or 'default'}")


Configuration:
  Use custom window: False
  Max tickers: 3


In [6]:
# Load summary with custom or default parameters
# Note: Default is top 10 articles per ticker (configurable via articles_per_ticker)
if USE_CUSTOM_WINDOW:
    summary = service.load_custom_summary(
        window_start=CUSTOM_WINDOW_START,
        window_end=CUSTOM_WINDOW_END,
        max_tickers=MAX_TICKERS,
    )
else:
    # Use default window but allow custom max_tickers
    if MAX_TICKERS is not None:
        # Get default window by loading previous day summary first
        default_summary = service.load_previous_day_summary()
        summary = service.load_custom_summary(
            window_start=default_summary.window_start,
            window_end=default_summary.window_end,
            max_tickers=MAX_TICKERS,
        )
    else:
        summary = service.load_previous_day_summary()

print(f"Summary loaded:")
print(f"  Window: {summary.window_start} to {summary.window_end}")
print(f"  Tickers: {len(summary.tickers)}")
print(f"  Total mentions: {summary.total_mentions}")
print(f"  Total articles: {summary.total_ranked_articles}")
print(f"  Articles per ticker: {service._articles_per_ticker} (top articles by engagement score)")
summary

Summary loaded:
  Window: 2025-11-21 12:00:00+00:00 to 2025-11-22 00:00:00+00:00
  Tickers: 3
  Total mentions: 73
  Total articles: 30
  Articles per ticker: 10 (top articles by engagement score)


DailySummaryResult(window_start=datetime.datetime(2025, 11, 21, 12, 0, tzinfo=datetime.timezone.utc), window_end=datetime.datetime(2025, 11, 22, 0, 0, tzinfo=datetime.timezone.utc), tickers=[DailyTickerSummary(ticker='BTC', mentions=28, articles=[DailySummaryArticle(article_id=464769, ticker='BTC', title='Comment in: Daily Discussion Thread for November 21, 2025', url='https://www.reddit.com/r/wallstreetbets/comments/1p2wt3n/daily_discussion_thread_for_november_21_2025/nq0egts/', text='#**BTC IS AN INVESTMENT** 🫵🏼🤡', published_at=datetime.datetime(2025, 11, 21, 12, 27, 3, tzinfo=datetime.timezone.utc), upvotes=23, num_comments=0, engagement_score=1.5572463768704934, confidence=0.7, source='reddit_comment', matched_terms=('BTC',), sentiment=None, subreddit='wallstreetbets', author='daviper87'), DailySummaryArticle(article_id=464826, ticker='BTC', title='Comment in: Daily Discussion Thread for November 21, 2025', url='https://www.reddit.com/r/wallstreetbets/comments/1p2wt3n/daily_discuss

In [70]:
# View the prompt that will be sent to the LLM
prompt = service.build_prompt(summary)
print("=" * 80)
print("PROMPT:")
print("=" * 80)
print(prompt)
print("=" * 80)
print(f"\nPrompt length: {len(prompt)} characters")


PROMPT:
You are an expert social networks and financial analyst specializing in assessing market sentiment and ambience around stocks based on articles from various sources including Reddit, news outlets, and other social media platforms. Your role is to analyze the collective sentiment, identify key themes and catalysts, and provide insights into how retail investors and the broader market perceive different stocks.

Analyze retail investor sentiment from 2025-11-06 12:00 UTC through 2025-11-07 00:00 UTC.
Provide two concise paragraphs per ticker: one covering momentum and sentiment, another capturing the most cited catalysts. End with a single market-wide takeaway.

Use the engagement-weighted highlights below. Articles are sorted by engagement score.

Ticker: META — 136 mentions

Subreddit: r/wallstreetbets
 - Score 2.06:
   META, you piece of shit. go back up
 - Score 1.75:
   I regret buying the $META dip last week. That was not the dip
 - Score 1.70:
   I’m down 110k on meta and 

In [ ]:
service.bui

In [35]:
# View the full LangChain payload (API key will be masked for display)
import json
from copy import deepcopy

payload = service.build_langchain_payload(summary)

# Create a display-safe version with masked API key
display_payload = deepcopy(payload)
if "llm" in display_payload and "api_key" in display_payload["llm"]:
    api_key = display_payload["llm"]["api_key"]
    masked_key = f"{api_key[:7]}...{api_key[-4:]}" if len(api_key) > 11 else "***"
    display_payload["llm"]["api_key"] = masked_key

print("=" * 80)
print("LANGCHAIN PAYLOAD STRUCTURE:")
print("=" * 80)
print(json.dumps(display_payload, indent=2, default=str))
print("=" * 80)

# Show payload components separately
print("\n" + "=" * 80)
print("PAYLOAD COMPONENTS:")
print("=" * 80)
print(f"Framework: {payload['framework']}")
print(f"Model: {payload['llm']['model']}")
print(f"API Key: {masked_key}")
print(f"Prompt length: {len(payload['prompt'])} characters")
print(f"Metadata keys: {list(payload['metadata'].keys())}")
print(f"  - Window: {payload['metadata']['window_start']} to {payload['metadata']['window_end']}")
print(f"  - Total mentions: {payload['metadata']['total_mentions']}")
print(f"  - Total articles: {payload['metadata']['total_ranked_articles']}")
print(f"  - Tickers: {len(payload['metadata']['tickers'])}")
print("=" * 80)

# Store the actual payload for manual use
actual_payload = payload


LANGCHAIN PAYLOAD STRUCTURE:
{
  "framework": "langchain",
  "llm": {
    "provider": "openai",
    "model": "gpt-4.1-mini",
    "api_key": "sk-proj...-u0A"
  },
  "prompt": "You are Market Pulse's summarization analyst.\nSummarize retail investor sentiment from 2025-11-05 12:00 UTC through 2025-11-06 00:00 UTC.\nProvide two concise paragraphs per ticker: one covering momentum and sentiment, another capturing the most cited catalysts. End with a single market-wide takeaway.\nUse the engagement-weighted highlights below.\n\nTicker: HOOD \u2014 154 mentions\n\nSubreddit: r/wallstreetbets\n - Score 1.70 (source reddit_comment, upvotes 31, comments 0, confidence 0.70, terms [HOOD]):\n   Fuck it HOOD 170c\n - Score 1.58 (source reddit_comment, upvotes 24, comments 0, confidence 0.70, terms [HOOD]):\n   Actually tho this HOOD earnings call is bullish af\n - Score 1.42 (source reddit_comment, upvotes 17, comments 0, confidence 0.70, terms [HOOD]):\n   LMFAO how in the world are HOOD and QCOM 

In [36]:
# View the metadata separately (useful for debugging)
print("=" * 80)
print("METADATA (Full Summary Data):")
print("=" * 80)
print(json.dumps(actual_payload["metadata"], indent=2, default=str))
print("=" * 80)


METADATA (Full Summary Data):
{
  "window_start": "2025-11-05T12:00:00+00:00",
  "window_end": "2025-11-06T00:00:00+00:00",
  "total_mentions": 154,
  "total_ranked_articles": 5,
  "tickers": [
    {
      "ticker": "HOOD",
      "mentions": 154,
      "articles": [
        {
          "article_id": 93028,
          "title": "Comment in: What Are Your Moves Tomorrow, November 06, 2025",
          "url": "https://www.reddit.com/r/wallstreetbets/comments/1opez1g/what_are_your_moves_tomorrow_november_06_2025/nnb0kl4/",
          "text": "Fuck it HOOD 170c",
          "published_at": "2025-11-05T20:59:57+00:00",
          "upvotes": 31,
          "num_comments": 0,
          "engagement_score": 1.6982105923718658,
          "confidence": 0.7,
          "source": "reddit_comment",
          "matched_terms": [
            "HOOD"
          ],
          "sentiment": null,
          "subreddit": "wallstreetbets",
          "author": "Dongkey_kong"
        },
        {
          "article_id": 93

In [64]:
# Manually initialize the LangChain model (for custom invocation)
from langchain.chat_models import init_chat_model
from app.config import settings

model_name = actual_payload["llm"]["model"]
api_key = actual_payload["llm"]["api_key"]

# Initialize the model manually
manual_model = init_chat_model(
    model_name,
    temperature=settings.daily_summary_llm_temperature,
    timeout=settings.daily_summary_llm_timeout_seconds,
    max_tokens=settings.daily_summary_llm_max_tokens,
    api_key=api_key,
)

print(f"Model initialized: {model_name}")
print(f"Temperature: {settings.daily_summary_llm_temperature}")
print(f"Max tokens: {settings.daily_summary_llm_max_tokens}")
print(f"Timeout: {settings.daily_summary_llm_timeout_seconds}s")


Model initialized: gpt-4.1-mini
Temperature: 0.7
Max tokens: 1000
Timeout: 30s


## Test New Structured Output with Sentiment

The following cells test the new `generate_langchain_summary` method which uses structured output to get both summary and sentiment classification.


In [12]:
# Show the input prompt for each ticker (what goes into the model)
from app.services.daily_summary import DailyTickerSummary


print("=" * 80)
print("INPUT PROMPTS FOR EACH TICKER:")
print("=" * 80)

ticker_prompts = []
for idx, ticker_summary in enumerate[DailyTickerSummary](summary.tickers, start=1):
    prompt = service.build_prompt_for_ticker(
        ticker_summary, summary.window_start, summary.window_end
    )
    ticker_prompts.append(prompt)
    
    print(f"\n{'=' * 80}")
    print(f"TICKER {idx}: {ticker_summary.ticker} ({ticker_summary.mentions} mentions)")
    print(f"{'=' * 80}")
    print(prompt)
    print(f"\nPrompt length: {len(prompt)} characters")
    print(f"Articles: {len(ticker_summary.articles)}")

print(f"\n{'=' * 80}")
print(f"Total tickers: {len(ticker_prompts)}")
print(f"{'=' * 80}")


INPUT PROMPTS FOR EACH TICKER:

TICKER 1: CRWV (100 mentions)
You are an expert social networks and financial analyst specializing in assessing market sentiment and ambience around stocks based on articles from various sources including Reddit, news outlets, and other social media platforms. Your role is to analyze the collective sentiment, identify key themes and catalysts, and provide insights into how retail investors and the broader market perceive different stocks.

Analyze retail investor sentiment for CRWV during the last day - November 11th.
Analyze the provided articles and synthesize a comprehensive summary. Your response must be structured as JSON with 'summary' and 'sentiment' fields.

Summary: Write two concise paragraphs - the first should cover momentum and overall sentiment trends, the second should capture the most cited catalysts and key themes.

Sentiment: Classify the overall retail investor sentiment by analyzing the collective tone, language patterns, and emotiona

In [13]:
# Test the new structured output method (returns SummaryInfo objects)
print("=" * 80)
print("TESTING STRUCTURED OUTPUT METHOD:")
print("=" * 80)

try:
    structured_responses = service.generate_langchain_summary(summary)
    
    print(f"\nReceived {len(structured_responses)} structured responses\n")
    
    for idx, (ticker_summary, response) in enumerate(zip(summary.tickers, structured_responses), start=1):
        print(f"{'=' * 80}")
        print(f"TICKER {idx}: {ticker_summary.ticker}")
        print(f"{'=' * 80}")
        print(f"\nSummary:")
        print(response.summary)
        print(f"\nSentiment: {response.sentiment}")
        print(f"Sentiment value: {response.sentiment.value if response.sentiment else None}")
        print()
        
except Exception as e:
    print(f"Error: {e}")
    print(f"Error type: {type(e).__name__}")
    import traceback
    traceback.print_exc()


TESTING STRUCTURED OUTPUT METHOD:

Received 3 structured responses

TICKER 1: CRWV

Summary:
The momentum for CRWV on November 11th was notably weak, with the implied move expected at 16% contrasting sharply with an actual price movement of just 0.28%. The overall sentiment among retail investors was predominantly negative, reflecting skepticism and bearish outlooks. Discussions heavily referenced the CEO's statement that the company will not achieve profitability until 2030, which dampened enthusiasm and led to calls for selling. Many comments expressed frustration and disappointment, with some users anticipating significant losses or tanking prices at the open. Despite some comparisons to peers like PLTR, the prevailing attitude was one of caution and pessimism.

Sentiment: LLMSentimentCategory.BEARISH
Sentiment value: Bearish

TICKER 2: BYND

Summary:
Retail investor sentiment around BYND on November 11th is overwhelmingly negative, with a strong bearish momentum evident in social m

In [17]:
response.summary



'RKLB has experienced a strong surge in retail investor enthusiasm on November 11th, as reflected by multiple high-engagement posts on r/wallstreetbets praising the stock\'s performance and calling it ready for significant upward movement. The overall sentiment is highly optimistic, with expressions such as "READY FOR LIFTOFF" and "RKLB Smoked earnings!" indicating that investors are reacting positively to recent developments. The momentum is clearly bullish, with frequent use of rocket and celebratory emojis emphasizing excitement and confidence in the stock\'s near-term potential.\n\nKey catalysts driving this positive sentiment include RKLB\'s recent earnings report, which appears to have exceeded investor expectations, as suggested by phrases like "Nice surprise" and "Smoked earnings." The stock is perceived as being "in play tomorrow," highlighting anticipation of continued trading activity and potential gains. The recurring themes focus on strong earnings performance and the expe

In [14]:
# Test manual invocation to get RAW responses (before parsing)
# This shows what the model actually returns before structured output parsing
print("=" * 80)
print("TESTING RAW RESPONSES (Manual Invocation):")
print("=" * 80)

from app.services.daily_summary import parse_llm_response

raw_responses = []
parsed_responses = []

for idx, (ticker_summary, prompt) in enumerate(zip(summary.tickers, ticker_prompts), start=1):
    print(f"\n{'=' * 80}")
    print(f"TICKER {idx}: {ticker_summary.ticker}")
    print(f"{'=' * 80}")
    
    try:
        # Invoke model without structured output to get raw response
        raw_response = manual_model.invoke(prompt)
        raw_content = raw_response.content if hasattr(raw_response, 'content') else str(raw_response)
        raw_responses.append(raw_content)
        
        print(f"\n--- RAW RESPONSE (Original) ---")
        print(raw_content)
        print(f"\nRaw response length: {len(raw_content)} characters")
        
        # Parse the raw response
        parsed = parse_llm_response(raw_content)
        parsed_responses.append(parsed)
        
        print(f"\n--- PARSED RESPONSE ---")
        print(f"Summary: {parsed.summary[:200]}..." if len(parsed.summary) > 200 else f"Summary: {parsed.summary}")
        print(f"Sentiment: {parsed.sentiment}")
        print(f"Sentiment value: {parsed.sentiment.value if parsed.sentiment else None}")
        
        # Show usage metadata if available
        if hasattr(raw_response, 'usage_metadata'):
            print(f"\n--- USAGE METADATA ---")
            print(raw_response.usage_metadata)
            
    except Exception as e:
        print(f"Error for {ticker_summary.ticker}: {e}")
        print(f"Error type: {type(e).__name__}")
        import traceback
        traceback.print_exc()
        raw_responses.append(None)
        parsed_responses.append(None)


TESTING RAW RESPONSES (Manual Invocation):

TICKER 1: CRWV
Error for CRWV: name 'manual_model' is not defined
Error type: NameError

TICKER 2: BYND
Error for BYND: name 'manual_model' is not defined
Error type: NameError

TICKER 3: RKLB
Error for RKLB: name 'manual_model' is not defined
Error type: NameError


Traceback (most recent call last):
  File "/var/folders/tn/gm6j130s3vzcrb5cjfbzl9z40000gn/T/ipykernel_41252/3135763440.py", line 19, in <module>
    raw_response = manual_model.invoke(prompt)
                   ^^^^^^^^^^^^
NameError: name 'manual_model' is not defined
Traceback (most recent call last):
  File "/var/folders/tn/gm6j130s3vzcrb5cjfbzl9z40000gn/T/ipykernel_41252/3135763440.py", line 19, in <module>
    raw_response = manual_model.invoke(prompt)
                   ^^^^^^^^^^^^
NameError: name 'manual_model' is not defined
Traceback (most recent call last):
  File "/var/folders/tn/gm6j130s3vzcrb5cjfbzl9z40000gn/T/ipykernel_41252/3135763440.py", line 19, in <module>
    raw_response = manual_model.invoke(prompt)
                   ^^^^^^^^^^^^
NameError: name 'manual_model' is not defined


In [ ]:
# Compare structured output vs parsed raw responses
print("=" * 80)
print("COMPARISON: Structured Output vs Parsed Raw Response")
print("=" * 80)

if len(structured_responses) == len(parsed_responses):
    for idx, (ticker_summary, structured, parsed) in enumerate(
        zip(summary.tickers, structured_responses, parsed_responses), start=1
    ):
        print(f"\n{'=' * 80}")
        print(f"TICKER {idx}: {ticker_summary.ticker}")
        print(f"{'=' * 80}")
        
        print(f"\n--- STRUCTURED OUTPUT ---")
        print(f"Summary length: {len(structured.summary)} chars")
        print(f"Sentiment: {structured.sentiment} ({structured.sentiment.value if structured.sentiment else None})")
        
        print(f"\n--- PARSED RAW RESPONSE ---")
        if parsed:
            print(f"Summary length: {len(parsed.summary)} chars")
            print(f"Sentiment: {parsed.sentiment} ({parsed.sentiment.value if parsed.sentiment else None})")
            
            # Compare
            print(f"\n--- COMPARISON ---")
            sentiment_match = structured.sentiment == parsed.sentiment
            summary_similarity = "Similar" if abs(len(structured.summary) - len(parsed.summary)) < 50 else "Different lengths"
            print(f"Sentiment match: {sentiment_match}")
            print(f"Summary: {summary_similarity}")
        else:
            print("No parsed response available")
else:
    print(f"Mismatch: {len(structured_responses)} structured vs {len(parsed_responses)} parsed")


In [ ]:
# Store responses for further analysis
# You can access:
# - ticker_prompts: List of prompts sent to the model
# - structured_responses: List of SummaryInfo objects from structured output
# - raw_responses: List of raw string responses from manual invocation
# - parsed_responses: List of ParsedLLMResponse objects from parsing raw responses

print("Variables available for analysis:")
print(f"  - ticker_prompts: {len(ticker_prompts)} prompts")
print(f"  - structured_responses: {len(structured_responses)} responses")
print(f"  - raw_responses: {len(raw_responses)} raw responses")
print(f"  - parsed_responses: {len(parsed_responses)} parsed responses")
print(f"\nExample access:")
print(f"  ticker_prompts[0]  # First ticker's prompt")
print(f"  structured_responses[0]  # First ticker's structured response")
print(f"  raw_responses[0]  # First ticker's raw response")
print(f"  parsed_responses[0]  # First ticker's parsed response")


In [ ]:
# Manually invoke the model with custom prompt (optional)
# Uncomment and modify the prompt below to test custom prompts

# custom_prompt = "Your custom prompt here"
# response = manual_model.invoke(custom_prompt)
# print("Response:")
# print(response.content if hasattr(response, 'content') else response)


In [39]:
# This cell is now redundant - prompt is shown in cell above
# Keeping for backward compatibility, but use the dedicated prompt cell instead
# prompt = service.build_prompt(summary)
# print(prompt)

In [65]:
print(prompt)


You are an expert social networks and financial analyst specializing in assessing market sentiment and ambience around stocks based on articles from various sources including Reddit, news outlets, and other social media platforms. Your role is to analyze the collective sentiment, identify key themes and catalysts, and provide insights into how retail investors and the broader market perceive different stocks.

Analyze retail investor sentiment from 2025-11-06 12:00 UTC through 2025-11-07 00:00 UTC.
Provide two concise paragraphs per ticker: one covering momentum and sentiment, another capturing the most cited catalysts. End with a single market-wide takeaway.

Use the engagement-weighted highlights below. Articles are sorted by engagement score.

Ticker: META — 136 mentions

Subreddit: r/wallstreetbets
 - Score 2.06:
   META, you piece of shit. go back up
 - Score 1.75:
   I regret buying the $META dip last week. That was not the dip
 - Score 1.70:
   I’m down 110k on meta and I bought

In [66]:
# Option 1: Use the service method (handles errors automatically)
# Uncomment to use:
# responses = service.generate_langchain_summary(summary)
# for idx, response in enumerate(responses, start=1):
#     print(f'Response {idx}:\n{response}\n')

# Option 2: Manually invoke the model (more control)
# Use the manual_model from the cell above
try:
    response = manual_model.invoke(prompt)
    print("=" * 80)
    print("LLM RESPONSE:")
    print("=" * 80)
    if hasattr(response, 'content'):
        print(response.content)
    else:
        print(response)
    print("=" * 80)
except Exception as e:
    print(f"Error: {e}")
    print(f"Error type: {type(e).__name__}")
    import traceback
    traceback.print_exc()

LLM RESPONSE:
**META Momentum and Sentiment:**  
Retail investor sentiment on META during the analyzed timeframe is overwhelmingly negative, reflecting significant frustration and financial pain among traders. The stock experienced a sharp decline, with mentions of a 22% drop over the past week dominating the discourse. Many retail investors expressed regret over recent purchases, describing the post-earnings dip as a false bottom and lamenting substantial unrealized losses, including individual reports of losses exceeding $100K. Despite some fleeting mentions of short-term rallies, the dominant tone is one of dismay and exhaustion, with a strong sense of capitulation evident in the language used across social media, particularly on r/wallstreetbets.

**META Key Catalysts:**  
The primary catalysts driving this negative sentiment are META’s recent earnings report and the subsequent price action, which failed to meet retail investors’ expectations. The overnight price volatility—specifi

In [67]:
response.usage_metadata

{'input_tokens': 422,
 'output_tokens': 350,
 'total_tokens': 772,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

In [7]:
summary

DailySummaryResult(window_start=datetime.datetime(2025, 11, 20, 12, 0, tzinfo=datetime.timezone.utc), window_end=datetime.datetime(2025, 11, 21, 0, 0, tzinfo=datetime.timezone.utc), tickers=[DailyTickerSummary(ticker='NVDA', mentions=64, articles=[DailySummaryArticle(article_id=469886, ticker='NVDA', title='What a crazy trap set for all $NVDA bulls', url='https://reddit.com/r/wallstreetbets/comments/1p28edd/what_a_crazy_trap_set_for_all_nvda_bulls/', text='Nasdaq 600 points tumble in just over an hour. All the Nvidia result bull positions getting hammered.', published_at=datetime.datetime(2025, 11, 20, 16, 57, 17, tzinfo=datetime.timezone.utc), upvotes=9473, num_comments=1166, engagement_score=4.264036014062277, confidence=0.5, source='reddit_post', matched_terms=('$NVDA',), sentiment=None, subreddit='wallstreetbets', author='Lumiaman88'), DailySummaryArticle(article_id=469888, ticker='NVDA', title='NVDA at open today vs. 3 hours into trading', url='https://reddit.com/r/wallstreetbets/

In [7]:
# Test build_prompt_for_ticker for a single ticker
print("=" * 80)
print("TESTING build_prompt_for_ticker:")
print("=" * 80)

if summary and summary.tickers:
    # Test with the first ticker
    sample_ticker = summary.tickers[0]
    sample_prompt = service.build_prompt_for_ticker(
        sample_ticker, summary.window_start, summary.window_end
    )
    
    print(f"\n{'=' * 80}")
    print(f"SAMPLE PROMPT FOR {sample_ticker.ticker} ({sample_ticker.mentions} mentions):")
    print(f"{'=' * 80}")
    print(sample_prompt)
    print(f"{'=' * 80}")
    print(f"\nPrompt length: {len(sample_prompt)} characters")
    print(f"Articles included: {len(sample_ticker.articles)}")
    print(f"{'=' * 80}\n")
else:
    print("No tickers available in summary")


TESTING build_prompt_for_ticker:

SAMPLE PROMPT FOR BTC (28 mentions):
You are an expert social networks and financial analyst specializing in assessing market sentiment and ambience around stocks based on articles from various sources including Reddit, news outlets, and other social media platforms. Your role is to analyze the collective sentiment, identify key themes and catalysts, and provide insights into how retail investors and the broader market perceive different stocks.

Analyze retail investor sentiment for BTC during the previous trading window (November 21, 2025 07:00 AM EST to November 21, 2025 07:00 PM EST). The summary must reflect discussions specific to November 22nd, 2025.
Analyze the provided articles and synthesize a comprehensive summary. Your response must be structured as JSON with 'summary' and 'sentiment' fields.

Summary: Write two concise paragraphs - the first should cover momentum and overall sentiment trends, the second should capture the most cited cataly

In [ ]:
# Don't forget to close the session when finished
session.close()